In [6]:
! pip install -r requirements.txt

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.1 MB 9.4 MB/s eta 0:00:01
   --------------- ------------------------ 0.8/2.1 MB 10.4 MB/s eta 0:00:01
   ------------------- -------------------- 1.0/2.1 MB 7.0 MB/s eta 0:00:01
   --------------------------- ------------ 1.4/2.1 MB 7.6 MB/s eta 0:00:01
   ----------------------------------- ---- 1.9/2.1 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------  2.1/2.1 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.5 MB/s eta 0:00:00


DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [8]:
import streamlit as st
import openai
import requests
import langchain
from langchain.document_loaders.excel import UnstructuredExcelLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import pickle

print('Requried Libraries Imported!')

Requried Libraries Imported!


In [ ]:
# Load OpenAI API key from .config file
def load_api_key():
    with open('.config', 'r') as file:
        return file.readline().strip()

openai.api_key = load_api_key()

In [ ]:
def fetch_article_content(url):
    loader = UnstructuredURLLoader(url)
    return loader.load()

def generate_embeddings(text):
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(text)
    embeddings = OpenAIEmbeddings()
    return embeddings.embed_documents(texts)

def save_faiss_index(index, path):
    with open(path, 'wb') as f:
        pickle.dump(index, f)

def load_faiss_index(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

st.title("Scheme Research Application")

urls = st.sidebar.text_area("Enter URLs (one per line):")

if st.sidebar.button("Process URLs"):
    urls = urls.split("\n")
    all_texts = []
    for url in urls:
        st.write(f"Fetching content from: {url}")
        content = fetch_article_content(url)
        all_texts.append(content)
    
    combined_text = "\n".join(all_texts)
    st.write("Generating embeddings...")
    embedding_vectors = generate_embeddings(combined_text)
    
    st.write("Indexing with FAISS...")
    faiss_index = FAISS.from_documents(embedding_vectors)
    save_faiss_index(faiss_index, "faiss_store_openai.pkl")
    st.success("Processing complete and FAISS index saved.")

if st.sidebar.button("Ask Question"):
    faiss_index = load_faiss_index("faiss_store_openai.pkl")
    question = st.text_input("Enter your question:")
    if question:
        st.write("Fetching answer...")
        similar_docs = faiss_index.similarity_search(question)
        answer = similar_docs[0]['text']
        st.write(f"Answer: {answer}")
        st.write(f"Source URL: {similar_docs[0]['metadata']['source']}")
